In [1]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import pandas as pd
import numpy as np

import optuna
from objective import Objective

#### 再現のための乱数値指定

In [2]:
SEED = 334

#### 学習データの読み込み

In [3]:
boston = load_boston()
df_X = pd.DataFrame(boston.data, columns = boston.feature_names, dtype = float)
df_y = pd.Series(boston.target, name = 'PRICE', dtype = float)

#### データの分割

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size = 0.2, random_state = SEED)

#### インスタンス化

In [5]:
objective = Objective(RandomForestRegressor, X_train, y_train, random_state = SEED, cv = 5)

#### 最適化

In [6]:
study = optuna.create_study()
study.optimize(objective, n_trials=10)

[I 2020-07-23 18:18:30,074] Finished trial#0 with value: -0.8350835563401929 with parameters: {'min_samples_split': 14, 'max_depth': 457, 'n_estimators': 426}. Best is trial#0 with value: -0.8350835563401929.
[I 2020-07-23 18:18:31,749] Finished trial#1 with value: -0.8385140172913257 with parameters: {'min_samples_split': 11, 'max_depth': 349, 'n_estimators': 233}. Best is trial#1 with value: -0.8385140172913257.
[I 2020-07-23 18:18:32,031] Finished trial#2 with value: -0.8282074447559358 with parameters: {'min_samples_split': 16, 'max_depth': 32, 'n_estimators': 34}. Best is trial#1 with value: -0.8385140172913257.
[I 2020-07-23 18:18:35,147] Finished trial#3 with value: -0.8452311544730872 with parameters: {'min_samples_split': 4, 'max_depth': 68, 'n_estimators': 396}. Best is trial#3 with value: -0.8452311544730872.
[I 2020-07-23 18:18:37,748] Finished trial#4 with value: -0.8436456047752265 with parameters: {'min_samples_split': 6, 'max_depth': 370, 'n_estimators': 345}. Best is t

#### 最適モデルを得る

In [7]:
best_clf = objective.clf(**study.best_params, **objective.fixed_params)

#### スコアで性能評価

In [8]:
best_clf.fit(X_train, y_train)
y_pred_on_test = best_clf.predict(X_test)
print(r2_score(y_test, y_pred_on_test))

0.8634940676676572
